# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846891463693                   -0.70    4.5         
  2   -7.852325402545       -2.26       -1.53    1.0   30.1ms
  3   -7.852615309268       -3.54       -2.56    1.5   33.4ms
  4   -7.852645990736       -4.51       -2.87    2.5   40.9ms
  5   -7.852646479883       -6.31       -3.14    1.0   29.8ms
  6   -7.852646679077       -6.70       -4.03    1.0   29.6ms
  7   -7.852646686530       -8.13       -5.24    2.0   36.1ms
  8   -7.852646686725       -9.71       -5.52    2.2   38.2ms
  9   -7.852646686725   +  -13.02       -5.51    1.2   30.9ms
 10   -7.852646686729      -11.37       -6.08    1.0   29.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846815155123                   -0.70           4.5         
  2   -7.852528862776       -2.24       -1.64   0.80    2.2    313ms
  3   -7.852636852984       -3.97       -2.73   0.80    1.0   28.3ms
  4   -7.852646512498       -5.02       -3.28   0.80    2.2   37.6ms
  5   -7.852646674347       -6.79       -4.14   0.80    1.2   29.3ms
  6   -7.852646686570       -7.91       -4.81   0.80    2.0   36.3ms
  7   -7.852646686724       -9.81       -5.52   0.80    1.2   30.1ms
  8   -7.852646686729      -11.24       -6.65   0.80    2.0   35.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.410660e+01     3.693004e+00
 * time: 0.07213401794433594
     1     1.655654e+00     1.730441e+00
 * time: 0.34137606620788574
     2    -1.749400e+00     2.076657e+00
 * time: 0.37036705017089844
     3    -3.816541e+00     1.736758e+00
 * time: 0.4119679927825928
     4    -5.394421e+00     1.474629e+00
 * time: 0.45406413078308105
     5    -6.885589e+00     7.639165e-01
 * time: 0.4963979721069336
     6    -6.926871e+00     1.188083e+00
 * time: 0.5251259803771973
     7    -7.558573e+00     6.852306e-01
 * time: 0.5547361373901367
     8    -7.614236e+00     8.567379e-01
 * time: 0.5842530727386475
     9    -7.764237e+00     5.397976e-01
 * time: 0.6139280796051025
    10    -7.805636e+00     4.341130e-01
 * time: 0.6435520648956299
    11    -7.824178e+00     1.018804e-01
 * time: 0.6731381416320801
    12    -7.836111e+00     6.754376e-02
 * time: 0.7025601863861084
    13    -7.846258e+00     6.944533e-02
 * time: 0.731288

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846916649413                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645925390                   -1.64         
  2   -7.852646686730       -6.12       -3.71    2.41s
  3   -7.852646686730      -13.34       -7.26    160ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 9.610405889860694e-7
|ρ_newton - ρ_scfv| = 4.016880975240788e-7
|ρ_newton - ρ_dm|   = 1.4424659963979763e-9
